In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import scipy
import re

#Data handling
from sklearn.model_selection import train_test_split

# Models 
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Normalizer
from sklearn.preprocessing import MinMaxScaler

#Pickling
from six.moves import cPickle as pickle

#Metrics
from sklearn.metrics import average_precision_score, recall_score, precision_score, f1_score

import matplotlib.pyplot as plt
import matplotlib.style as ms
import librosa.display
import IPython.display as ipd
import seaborn
ms.use('seaborn-muted')
%matplotlib inline

x_train = []
y_train = []
x_test = []
y_test = []

In [2]:
def get_gender(text):
    text = text[len(text)-4]
    if('F' in text):
        return 'F'
    elif('M' in text):
        return 'M'
    else:
        return 'N/A'
    
def get_word_segments(align_file):
    segments = []
    with open(align_file,'r') as openFile:
        read_lines = openFile.readlines()
        lines = read_lines[1:len(read_lines)-1]
        for line in lines:
            splitted = line.split()
            segments.append({'sf': splitted[0], 'ef':splitted[1],'seg_scr':splitted[2],'word':splitted[3]})
    return segments
    
def generate_IEMOCAP_df():
    root_path = 'data/IEMOCAP_full_release/'
    folders = os.listdir(root_path)
    session_folders = filter(lambda x: x.startswith('Session'),folders)
    sentences_wav = '/sentences/wav/'
    forced_alignment = '/sentences/ForcedAlignment/'
    dialog_transcriptions = '/dialog/transcriptions/' #Ses01F_impro01.txt
    dialog_emo = '/dialog/EmoEvaluation/' #Ses01F_impro01.txt
    data = {}
    emotions = {}
    alignment = {}
    for session in session_folders:
        temp_root = root_path + session
        temp_conversations_path = temp_root + sentences_wav
        temp_transcripts_path = temp_root + dialog_transcriptions
        temp_alignments_path = temp_root + forced_alignment
        temp_emo_path = temp_root + dialog_emo
        emo_eval = os.listdir(temp_emo_path)
        for emo_file in emo_eval:
            emo_file_path = temp_emo_path + emo_file
            if(emo_file.endswith('.txt')):
                with open(emo_file_path,'r') as openFile:
                    for line in openFile.readlines():
                        if(line.startswith('[')):
                            line = line.split()
                            key = line[3]
                            emotion = line[4]
                            valence = re.search('[0-9]+.[0-9]+', line[5]).group(0)
                            arousal = re.search('[0-9]+.[0-9]+',line[6]).group(0)
                            dominance = re.search('[0-9]+.[0-9]+',line[7]).group(0)
                            emotions[key] = [emotion,valence,arousal, dominance]
        transcriptions = os.listdir(temp_transcripts_path)
        for transcript in transcriptions:
            transcript_path = temp_transcripts_path + transcript
            with open(transcript_path,'r') as openFile:
                for line in openFile.readlines():
                    if(line.startswith('Ses')):
                        line = line.split(' [')
                        key = line[0]
                        if('XX' not in key):
                            line = ' ['.join(line[1:])
                            line = line.split(']: ')
                            time = line[0]
                            script = ']: '.join(line[1:])
                            start_t, end_t = time.split('-')
                            data[key] = [start_t,end_t,script.split('\r\n')[0]]
        conversations = os.listdir(temp_conversations_path)
        for conversation in conversations:
            files_path = temp_conversations_path+ conversation
            alignments_path = temp_alignments_path + conversation
            files = os.listdir(files_path)
            for file in files:
                key,suf = file.split('.')
                if(suf == 'wav'):
                    data[key].append(files_path + '/'+ file)
                    data[key].append(alignments_path + '/'+key+'.wdseg')
    sentence_df = pd.DataFrame(data,index=['start_time','end_time','text','wav_path','alignment_path']).T
    emotion_df = pd.DataFrame(emotions, index=['emotion','valence','arousal','dominance']).T
    full = sentence_df.join(emotion_df)
    full = full.reset_index()
    full['gender'] = full['index'].apply(get_gender)
    return full

In [3]:
def log_reg(x_train,y_train,x_test,y_test, delete = True):
    logreg = LogisticRegression()
    logreg.fit(x_train, y_train)
    Y_pred = logreg.predict(x_test)
    acc_log = round(logreg.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_pred)
    recall = recall_score(y_test,Y_pred)
    if(delete):
        del logreg
        return Y_pred, acc_log, precision, recall
    return logreg,Y_pred,acc_log, precision, recall

# _, acc_log = log_reg(x_train,ang_train, x_test, ang_test)
# print(acc_log)

def svc(x_train,y_train,x_test,y_test, delete = True):
    svc = SVC()
    svc.fit(x_train,y_train)
    Y_svcpred = svc.predict(x_test)
    acc_svc = round(svc.score(x_test, y_test) * 100,2)
    precision = precision_score(y_test, Y_svcpred)
    recall = recall_score(y_test,Y_svcpred)
    if(delete):
        del svc 
        return Y_svcpred, acc_svc,precision, recall
    return svc, Y_svcpred, acc_svc,precision,recall

# _, acc_svc = svc(x_train,ang_train,x_test,ang_test)
# print(acc_svc)

def knn(x_train,y_train,x_test,y_test,n_neighbors = 3,delete = True):
    knn = KNeighborsClassifier(n_neighbors = n_neighbors)
    knn.fit(x_train, y_train)
    Y_knnpred = knn.predict(x_test)
    acc_knn = round(knn.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_knnpred)
    recall = recall_score(y_test,Y_knnpred)
    if(delete):
        del knn
        return Y_knnpred, acc_knn,precision,recall
    return knn, Y_knnpred, acc_knn, precision,recall
# _,acc_knn = knn(x_train,ang_train,x_test,ang_test)
# print(acc_knn)

def gaussian(x_train,y_train,x_test,y_test,delete = True):
    gaussian = GaussianNB()
    gaussian.fit(x_train, y_train)
    Y_gaussianpred = gaussian.predict(x_test)
    acc_gaussian = round(gaussian.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_gaussianpred)
    recall = recall_score(y_test,Y_gaussianpred)
    if(delete):
        del gaussian
        return Y_gaussianpred, acc_gaussian, precision,recall
    return gaussian, Y_gaussianpred, acc_gaussian,precision,recall

# _, acc_gaussian = gaussian(x_train,ang_train,x_test,ang_test)
# print(acc_gaussian)

def perceptron(x_train,y_train,x_test,y_test,delete = True):
    perceptron = Perceptron()
    perceptron.fit(x_train, y_train)
    Y_perceptronpred = perceptron.predict(x_test)
    acc_perceptron = round(perceptron.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_perceptronpred)
    recall = recall_score(y_test,Y_perceptronpred)
    if(delete):
        del perceptron
        return Y_perceptronpred, acc_perceptron,precision,recall
    return perceptron, Y_perceptronpred ,acc_perceptron,precision,recall
# _, acc_perceptron = perceptron(x_train,ang_train,x_test,ang_test)
# print(acc_perceptron)

def linear_svc(x_train,y_train,x_test,y_test,delete = True):
    linear_svc = LinearSVC()
    linear_svc.fit(x_train, y_train)
    Y_linearsvcpred = linear_svc.predict(x_test)
    acc_linear_svc = round(linear_svc.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_linearsvcpred)
    recall = recall_score(y_test,Y_linearsvcpred)
    if(delete):
        del linear_svc
        return Y_linearsvcpred, acc_linear_svc,precision,recall
    return linear_svc, Y_linearsvcpred, acc_linear_svc,precision,recall
# _, acc_linear_svc = linear_svc(x_train,ang_train,x_test,ang_test)
# print(acc_linear_svc)

def sgd(x_train,y_train,x_test,y_test,delete = True):
    sgd = SGDClassifier()
    sgd.fit(x_train, y_train)
    Y_linearsgdpred = sgd.predict(x_test)
    acc_sgd = round(sgd.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_linearsgdpred)
    recall = recall_score(y_test,Y_linearsgdpred)
    if(delete):
        del sgd
        return Y_linearsgdpred , acc_sgd,precision,recall
    return sgd, Y_linearsgdpred, acc_sgd,precision,recall
# _, acc_sgd = sgd(x_train,ang_train,x_test,ang_test)
# print(acc_sgd)

def decision_tree(x_train,y_train,x_test,y_test,delete = True):
    decision_tree = DecisionTreeClassifier()
    decision_tree.fit(x_train, y_train)
    Y_dectreepred = decision_tree.predict(x_test)
    acc_decision_tree = round(decision_tree.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_dectreepred)
    recall = recall_score(y_test,Y_dectreepred)
    if(delete):
        del decision_tree
        return Y_dectreepred , acc_decision_tree,precision,recall
    return decision_tree, Y_dectreepred , acc_decision_tree,precision,recall
# _, acc_decision_tree = decision_tree(x_train,ang_train,x_test,ang_test)
# print(acc_decision_tree)

def random_forest(x_train,y_train,x_test,y_test,n_estimators = 100, delete = True):
    random_forest = RandomForestClassifier(n_estimators= n_estimators)
    random_forest.fit(x_train, y_train)
    Y_pred = random_forest.predict(x_test)
    acc_random_forest = round(random_forest.score(x_test, y_test) * 100, 2)
    precision = precision_score(y_test, Y_pred)
    recall = recall_score(y_test,Y_pred)
    if(delete):
        del random_forest
        return Y_pred, acc_random_forest,precision,recall
    return random_forest, Y_pred, acc_random_forest,precision,recall
# _, acc_random_forest = random_forest(x_train,ang_train, x_test , ang_test)
# print(acc_random_forest)

def compare_models(x_train, y_train, x_test, y_test):
    _,acc_svc,pre_svc,rec_svc = svc(x_train,y_train,x_test,y_test)
    _,acc_knn,pre_knn,rec_knn = knn(x_train,y_train,x_test,y_test)
    _,acc_log,pre_log,rec_log = log_reg(x_train,y_train,x_test,y_test)
    _,acc_random_forest,pre_rf,rec_rf = random_forest(x_train,y_train,x_test,y_test)
    _,acc_gaussian,pre_gau,rec_gau = gaussian(x_train,y_train,x_test,y_test)
    _,acc_perceptron,pre_per,rec_per = perceptron(x_train,y_train,x_test,y_test)
    _,acc_sgd,pre_sgd,rec_sgd = sgd(x_train,y_train,x_test,y_test)
    _,acc_linear_svc,pre_lsvc,rec_lsvc = linear_svc(x_train,y_train,x_test,y_test)
    _,acc_decision_tree,pre_dtree,rec_dtree =  decision_tree(x_train,y_train,x_test,y_test)
    models = pd.DataFrame({
        'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
                  'Random Forest', 'Naive Bayes', 'Perceptron', 
                  'Stochastic Gradient Decent', 'Linear SVC', 
                  'Decision Tree'],
        'Score': [acc_svc, acc_knn, acc_log, 
                  acc_random_forest, acc_gaussian, acc_perceptron, 
                  acc_sgd, acc_linear_svc, acc_decision_tree],
        'Precision': [pre_svc, pre_knn,pre_log,
                      pre_rf,pre_gau,pre_per,pre_sgd,
                      pre_lsvc,pre_dtree],
        'Recall': [rec_svc,rec_knn,rec_log,rec_rf,rec_gau,rec_per,rec_sgd,rec_lsvc,rec_dtree]
    })
    return models #.sort_values(by='Score', ascending=False)

def map_for_emotion(y_list, emotional_mapping,emotion):
    return list(map(lambda x: 1 if x == emotional_mapping[emotion] else 0,y_list))

def compare_models_x_emotions(x_train, x_test, emotions):
    result = {'Model': ['Support Vector Machines', 'KNN', 'Logistic Regression', 
                  'Random Forest', 'Naive Bayes', 'Perceptron', 
                  'Stochastic Gradient Decent', 'Linear SVC', 
                  'Decision Tree']}
    for emotion,data in emotions.iteritems():
        y_train = data[0]
        y_test = data[1]
        _,acc_svc = svc(x_train,y_train,x_test,y_test)
        _,acc_knn = knn(x_train,y_train,x_test,y_test)
        _,acc_log = log_reg(x_train,y_train,x_test,y_test)
        _,acc_random_forest = random_forest(x_train,y_train,x_test,y_test)
        _,acc_gaussian = gaussian(x_train,y_train,x_test,y_test)
        _,acc_perceptron = perceptron(x_train,y_train,x_test,y_test)
        _,acc_sgd = sgd(x_train,y_train,x_test,y_test)
        _,acc_linear_svc = linear_svc(x_train,y_train,x_test,y_test)
        _,acc_decision_tree =  decision_tree(x_train,y_train,x_test,y_test)
        result[emotion] = [acc_svc, acc_knn, acc_log, 
                  acc_random_forest, acc_gaussian, acc_perceptron, 
                  acc_sgd, acc_linear_svc, acc_decision_tree]
    models = pd.DataFrame(result)
    models = models[['Model'] + list(emotions.iterkeys())]
    return models
        
    



In [4]:
# full = generate_IEMOCAP_df()
data = pd.read_csv('data/IEMOCAP_sentences.csv',index_col=0)
data = data[(data.emotion != 'oth')&(data.emotion != 'xxx')]
emotional_mapping = {'ang': 0, 'sad': 1, 'exc': 2, 'neu': 3,'fru': 4,'hap': 5,'fea': 6,'sur': 7,'dis': 8, 'xxx':9,'oth':10}
data['emotion'] = data['emotion'].map( emotional_mapping ).astype(int)

In [5]:
def get_sample(data,emotional_mapping,emotion):
    emotion_code = emotional_mapping[emotion]
    emotion_sample = data[data.emotion == emotion_code]
    sample_size = len(emotion_sample)
    oth_emotion_sample = data[data.emotion != emotion_code].sample(n=sample_size)
    sample = pd.concat([oth_emotion_sample,emotion_sample]).sample(frac=1)
    y = map_for_emotion(sample.emotion,emotional_mapping,'ang')
    return sample,y

def get_mfcc_feature(filename,n_mfcc):
    y, sr = librosa.load(filename)
    #IPython.display.Audio(data=y,rate=sr)
    mfcc = librosa.feature.mfcc(y=y,sr=sr, n_mfcc=n_mfcc)
    print("Finished "+ filename)
    return np.mean(mfcc.T,axis=0)

def play_audio(filename):
    y, sr = librosa.load(filename)
    return ipd.Audio(data=y,rate=sr)

def get_confusion(y_pred, y_true):
    tp = []
    fp =[]
    fn =[]
    tn = []
    for i in range(len(y_pred)):
        if(y_pred[i] == y_true[i]):
            if y_pred[i] == 1:
                tp.append(True)
                tn.append(False)
            else:
                tn.append(True)
                tp.append(False)
            fp.append(False)
            fn.append(False)
        else:
            if(y_pred[i] == 1):
                fp.append(True)
                fn.append(False)
            else:
                fp.append(False)
                fn.append(True)
            tn.append(False)
            tp.append(False)
            
    return tp,tn,fp,fn
    

# Anger Binary Models

In [6]:
# x,y = get_sample(data,emotional_mapping,'ang')
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# try:
#     with open('pickles/anger/base_datasets.pickle','wb') as f:
#         save = {
#             'X_train' : X_train,
#             'X_test' : X_test,
#             'y_train': y_train,
#             'y_test' : y_test
#         }
#         pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
# except Exception as e:
#     print('Unable to save data to anger base datasets pickle: ', e)
    
try:
    with open('pickles/anger/base_datasets.pickle','rb') as f:
        save = pickle.load(f)
        X_train = save['X_train']
        X_test = save['X_test']
        y_train = save['y_train']
        y_test = save['y_test']
        del save
except Exception as e:
    print('Error loading base datasets pickle: ', e)

print X_train.shape, len(y_train)
print X_test.shape, len(y_test)

(1764, 11) 1764
(442, 11) 442


In [7]:


# x_train = list(map(lambda x: get_mfcc_feature(x,20),X_train['wav_path']))
# x_test = list(map(lambda x: get_mfcc_feature(x,20),X_test['wav_path']))
# try:
#     with open('pickles/anger/mfcc20.pickle','wb') as f:
#         save = {
#             'train_dataset' : x_train,
#             'train_labels' : y_train,
#             'test_dataset' : x_test,
#             'test_labels' : y_test
#         }
#         pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
# except Exception as e:
#     print('Unable to save data to mfcc pickle: ', e)

try:
    with open('pickles/anger/mfcc20.pickle','rb') as f:
        save = pickle.load(f)
        x_train = save['train_dataset']
        y_train = save['train_labels']
        x_test = save['test_dataset']
        y_test = save['test_labels']
        del save
except Exception as e:
    print('Error at:', e)

In [ ]:
compare_models(x_train, y_train,x_test, y_test)

In [ ]:
model, Y_pred,acc, precision, recall = log_reg(x_train,y_train,x_test,y_test, delete = False)

In [ ]:
tp,tn,fp,fn = get_confusion(Y_pred, y_test)

In [ ]:
X_test[['text','emotion']].iloc[fn].head(20)

In [ ]:
emotional_mapping = {'ang': 0, 'sad': 1, 'exc': 2, 'neu': 3,'fru': 4,'hap': 5,'fea': 6,'sur': 7,'dis': 8, 'xxx':9,'oth':10}

In [ ]:
play_audio(X_test.iloc[fn]['wav_path'].values[0])

# True Positives

In [ ]:
Y_pred[:5]

In [ ]:
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, Y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['Anger','NotAnger'],
                      title='Confusion matrix, without normalization')

# # Plot normalized confusion matrix
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=['Anger','NotAnger'], normalize=True,
#                       title='Normalized confusion matrix')

plt.show()